here original data is used to truncate the lists of time series columns and unlike previous attempts, failure columns are kept to be used as categorical columns.

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
original_data = pd.read_parquet("../../data/test_107_reduced.parquet")
original_data

In [ ]:
# removing a row that had unusually high number of time samples
data = original_data[original_data.doorcyclecounter != 3503]

Defining relevant column types

In [ ]:
failure_columns = [
	'cd_counter_rollers_installation',
	'ld_counter_rollers_installation',
	'closing_device_failure',
	'misalignment_cd_vs_ld',
	'misalignment_cd_vs_ld_sill_gap',
	'belt_tension',
	'pulley_is_touching_belt',
	'zero_position',
]

environment_columns = [
	'cmcouplerfriction',
	'cmdoorfriction',
	'cmelectronicage',
	'cmvibration',
]

static_state_columns = [
	'doorcyclecounter',
]

time_state_columns = [
	'doorforce',
	'doorspeed',
	'doorposition',
	# 'time',
]

In [ ]:
doorforce_data = data['doorforce']
doorforce_data.head(5).to_clipboard(sep=',')

In [ ]:
def sample_series(arrays, target_length):
    """
    Sample values from a series of arrays to produce arrays of equal length.
    
    Parameters:
    arrays (list of list of int/float): Series of arrays with different lengths.
    target_length (int): The length of the arrays after sampling.
    
    Returns:
    list of list of int/float: Series of arrays with the target length.
    """
    sampled_series = []

    for array in arrays:
        if len(array) == target_length:
            sampled_series.append(array)
        else:
            indices = np.linspace(0, len(array) - 1, target_length).astype(int)
            sampled_array = [array[i] for i in indices]
            sampled_series.append(sampled_array)
    
    return sampled_series

# Example usage
# arrays = [[1, 2, 3, 4, 5], [10, 20, 30], [100, 200, 300, 400, 500, 600]]
# target_length = 3
# sampled_series = sample_series(arrays, target_length)
# print(sampled_series)

In [ ]:

arrays = [[ -75.1  -75.1  -92.8  -73.5  -51.9  -46.3  -46.3  -38.3  -48.   -21.4
  -17.5   -9.3   -5.5   -3.4    6.1   20.9   34.4   51.6   77.7  113.6
  137.4  142.8  148.8  155.5  162.8  169.   171.7  170.8  172.7  176.9
  179.6  182.7  176.1  165.6  153.3  138.8  125.1  112.6   88.9   66.9
   45.5   30.7   30.3   38.5   49.8   60.4   65.5   67.8   69.4   75.5
   75.9   79.    83.4   86.    88.2   88.8   92.   131.   167.2  160.6
  137.4  118.    96.6   73.1   59.9   59.5   59.6   59.5   59.6   59.6
   59.6   59.5   59.5   59.6   59.6   59.6   59.6   59.5   59.5   59.6
   59.5   59.5   59.5   59.6   59.5   59.6   59.6   59.6   59.5   59.6
   59.6   59.6   59.5   59.6   59.6   59.6   59.5   59.6   59.6   59.5
   59.5   59.5   59.6   59.5   59.6   59.5   59.6   59.5   59.6   59.6
   74.9   83.1   77.5  -78.4   28.3   38.7   13.2    0.    -3.5   -3.1
   -2.2    2.     5.     7.9   11.    13.5   12.9   13.8   14.3   13.7
   13.7   13.1   13.4   14.    13.    13.1   11.4   11.4   12.2   12.4
   11.6   10.4   11.     8.7    8.2    7.8   12.5   13.2   12.2   11.2
   10.5    8.7    9.4   11.2   11.7   12.6   14.1   13.6   15.3   18.5
   29.2   47.2   56.9   44.    21.    15.8   16.4   15.1   -0.3  -23.2
  -35.9  -46.1  -52.8  -56.8 -117.4 -100.5  -84.8 -167.9  -89.   -77.5
 -136.   -97.9 -136.8 -150.9 -168.4 -165.  -170.  -161.6 -140.  -115.7
  -97.   -76.8  -75.1  -75.1  -75.1  -75.1  -75.1],[ -75.1  -75.1  -85.   -92.1  -52.8  -51.6  -51.6  -50.   -36.7  -44.2
  -11.5  -13.9   -6.3   -5.1    4.5   15.5   28.3   41.2   69.5  110.4
  140.2  145.6  146.5  151.3  156.   161.7  160.8  163.1  167.8  173.3
  171.8  176.3  179.2  177.   158.6  142.7  128.   115.8  102.8   77.6
   50.6   31.1   24.    32.1   42.8   57.6   67.2   66.9   72.8   76.1
   78.5   81.1   83.7   86.6   88.7   90.6  116.5  164.   161.1  142.5
  120.4   96.3   76.8   60.1   59.5   59.5   59.5   59.5   59.5   59.6
   59.5   59.5   59.5   59.5   59.5   59.6   59.5   59.5   59.5   59.5
   59.5   59.5   59.6   59.5   59.5   59.5   59.5   59.5   59.5   59.5
   59.5   59.5   59.5   59.6   59.5   59.6   59.5   59.5   59.5   59.5
   59.6   59.6   59.5   59.6   59.5   59.6   59.6   59.5   64.2   81.1
   76.3   20.9   -6.5  -17.4   -3.1   11.6   15.3   10.3    8.9   13.1
   14.    12.4   12.4   12.9   13.6   13.    13.3   13.6   14.6   15.3
   11.8    9.9   10.3   10.4   11.8   12.7   10.1    9.3    7.7    7.9
    7.6    9.    14.    14.9   12.9   10.7    9.    10.8   11.8   11.8
   12.3   12.1   12.7   12.5   14.1   18.9   32.4   53.6   55.3   40.
   14.7   15.4   14.8    6.7  -12.1  -39.5  -44.6  -52.1  -58.1  -65.1
  -85.1  -98.6 -140.9 -131.6  -90.8 -117.3 -123.6 -141.6 -162.2 -170.1
 -169.8 -170.2 -154.2 -136.  -114.1  -87.   -75.6  -75.1  -75.   -75.1
  -75.1  -75.1],[ -75.1  -75.1  -87.9  -88.3  -61.2  -51.1  -51.1  -50.2  -49.1  -31.7
  -16.   -12.8   -6.7   -4.2    4.1   17.4   30.8   40.6   70.5  107.7
  133.6  141.8  146.7  154.8  162.7  167.2  169.7  169.1  170.4  173.8
  182.1  181.8  176.2  168.4  153.2  140.4  125.7  114.5   95.6   67.3
   49.5   32.3   30.3   38.5   48.2   56.4   63.7   67.7   69.5   77.5
   76.8   79.7   83.1   84.2   87.    88.9   95.2  146.9  169.   154.1
  132.6  109.3   89.6   67.9   59.5   59.5   59.6   59.5   59.5   59.5
   59.5   59.5   59.5   59.5   59.5   59.5   59.5   59.5   59.5   59.5
   59.5   59.5   59.6   59.5   59.5   59.5   59.5   59.5   59.5   59.6
   59.5   59.5   59.6   59.5   59.6   59.5   59.5   59.6   59.5   59.5
   59.5   59.6   59.6   59.5   59.6   59.5   59.5   59.5   59.5   59.8
   79.8   81.9   46.     3.5   -9.7   -8.7   -3.3   12.7    8.8   11.3
   11.1   14.1   15.1   11.9   12.1   12.8   13.4   12.8   12.7   14.2
   15.8   13.9   11.8   10.4   10.    10.5   11.8   11.4   11.5    8.9
    7.8    7.1    7.9   11.6   14.3   12.9   12.3   11.4    9.3   10.
   11.4   11.7   14.8   13.6   13.5   13.9   19.7   31.    51.9   56.3
   36.1   19.5   17.3   16.3   14.6   -3.2  -30.5  -40.9  -47.6  -54.8
  -59.1 -127.7 -125.  -102.5  -91.2 -152.7 -108.8 -112.7 -122.1 -150.7
 -161.2 -168.3 -170.  -169.5 -151.9 -130.3 -106.2  -86.4  -75.3  -75.1
  -75.1  -75.1  -75.1],[ -75.1  -75.1  -85.6  -83.4  -57.2  -51.   -51.   -50.4  -33.1  -35.7
  -17.4  -14.3   -5.6   -5.     7.7   20.    33.6   47.    80.4  121.7
  138.3  143.7  148.2  155.4  165.4  175.3  173.6  168.5  176.4  177.9
  180.9  186.1  176.2  164.   145.8  133.4  124.   111.7   90.8   66.
   41.1   27.5   23.8   30.7   50.1   63.3   69.9   69.9   75.2   76.1
   78.3   83.4   86.2   87.8   90.5   92.1  129.9  166.4  161.7  136.9
  118.5   96.9   72.8   59.9   59.6   59.5   59.5   59.5   59.5   59.5
   59.5   59.5   59.5   59.5   59.5   59.5   59.5   59.5   59.5   59.5
   59.5   59.5   59.5   59.5   59.5   59.5   59.5   59.5   59.5   59.5
   59.5   59.5   59.5   59.5   59.5   59.5   59.5   59.5   59.5   59.5
   59.5   59.5   59.5   59.6   59.5   59.6   59.5   59.5   59.5   79.4
   80.6   45.5   -2.   -16.1   -7.2    3.    13.2   12.9    9.9    9.1
   13.5   12.5   13.2   14.4   13.    13.    12.8   14.8   16.1   13.9
   11.5   10.6    9.6   12.6   12.9   11.6   10.1    8.2   10.3    8.6
    7.9   10.2   13.6   14.5   13.3   10.5    8.4    9.4   11.2   11.5
   12.1   14.3   13.5    9.4   13.2   19.2   31.6   53.5   56.8   39.6
   17.    16.2   15.5    8.1   -7.   -33.6  -43.5  -52.4  -54.8  -59.4
 -100.8 -130.7 -132.5  -79.   -87.2 -125.  -118.5 -138.2 -159.6 -164.8
 -170.2 -170.3 -164.  -145.5 -123.9 -100.   -80.5  -75.1  -75.1  -75.1
  -75.1  -75.1],[ -75.1  -75.1  -78.5  -91.4  -59.5  -49.7  -49.7  -49.2  -37.8  -46.5
  -20.9  -13.    -6.3   -5.    -0.8   13.6   35.6   51.8   93.6  144.6
  142.4  146.1  145.6  153.6  157.9  164.5  163.6  166.3  164.4  172.1
  178.6  180.1  181.1  173.   152.5  136.9  128.2  108.    80.1   56.5
   38.5   30.6   36.2   44.2   55.7   59.4   64.9   67.6   72.8   78.2
   77.8   80.    84.6   86.8   88.1   89.2  103.1  143.2  169.1  149.6
  131.   109.3   85.    66.    59.6   59.5   59.6   59.6   59.5   59.6
   59.5   59.5   59.5   59.5   59.5   59.5   59.5   59.5   59.5   59.5
   59.5   59.5   59.5   59.6   59.5   59.5   59.5   59.5   59.6   59.5
   59.5   59.5   59.5   59.5   59.5   59.5   59.5   59.5   59.5   59.6
   59.5   59.5   59.6   59.5   59.6   59.5   59.5   59.6   59.5   80.5
   81.5   39.5   -1.5  -11.7   -6.8    4.9   16.5    9.4    8.    10.1
   13.8   14.5   13.8   12.6   13.6   13.4   12.4   12.2   13.2   15.5
   14.9   12.5   10.3    9.5   10.4   11.7   11.4    9.8    8.9    9.3
    8.3    9.    10.8   12.6   14.5   12.4   10.7    8.5   10.3   13.1
   14.4   14.9   12.5   11.7   12.1   16.4   25.4   42.6   56.8   47.1
   26.8   16.4   16.9   15.     9.    -6.8  -37.6  -43.4  -52.3  -55.4
  -60.  -105.6 -132.2 -100.8  -86.3 -125.9 -116.7 -136.5 -144.7 -161.2
 -163.5 -170.1 -170.2 -164.5 -140.8 -120.1  -98.8  -78.5  -75.1  -75.1
  -75.1  -75.1  -75.1]]
target_length = 3
sampled_series = sample_series(arrays, target_length)
print(sampled_series)

In [ ]:
sampled_doorforce = sample_series(data['doorforce'],20)
sampled_doorforce

In [ ]:
# Apply the downsampling to every element in the 'doorforce' column
downsampled_data = doorforce_data.apply(lambda x: x[::2])

# Find the minimum length of the downsampled arrays
min_length = downsampled_data.apply(len).min()

# Truncate each downsampled array to the minimum length
equal_length_doorforce = downsampled_data.apply(lambda x: x[:min_length])



In [ ]:
import matplotlib.pyplot as plt

for row_values in sampled_doorforce:
    plt.plot(row_values)
    
plt.xlabel('Index')
plt.ylabel('Value')
plt.title('Downsampled Doorforce Data')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

for row_values in equal_length_doorforce:
    plt.plot(row_values)
    
plt.xlabel('Index')
plt.ylabel('Value')
plt.title('Downsampled Doorforce Data')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

for row_values in doorforce_data:
    plt.plot(row_values)
    
plt.xlabel('Index')
plt.ylabel('Value')
plt.title('Doorforce Data')
plt.show()

In [ ]:
row_lengths = doorforce_data.apply(lambda x: len(x))
print(row_lengths)

In [ ]:
doorforce_series = data['doorforce'].apply(pd.Series)
doorspeed_series = data['doorspeed'].apply(pd.Series)
doorposition_series = data['doorposition'].apply(pd.Series)

doorforce_series = doorforce_series.add_prefix('doorforce_')
doorspeed_series = doorspeed_series.add_prefix('doorspeed_')
doorposition_series = doorposition_series.add_prefix('doorposition_')
doorforce_series


In [ ]:

truncated_data = data.copy()
truncated_data.drop(columns=['doorforce', 'doorspeed', 'doorposition'], inplace=True)
truncated_data


In [ ]:
train_data = pd.concat([truncated_data, doorforce_series, doorspeed_series, doorposition_series], axis=1)


In [ ]:
import matplotlib.pyplot as plt

# Plotting all values of doorforce_series
doorforce_series.plot()

# Adding labels and title
plt.xlabel('Index')
plt.ylabel('Door Force')
plt.title('Plot of Door Force Series')

# Display the plot
plt.show()

In [ ]:
a = train_data.ffill(axis=1)


In [ ]:
a.isna().sum()

In [ ]:
a.to_parquet("../../data/aaa.parquet")